In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [44]:
df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [45]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(395, 21)

In [46]:
dnn_model = tf.keras.models.load_model('dnn.h5')

In [47]:
def my_loss(real, X_pred):
    return dnn_model(X_pred)

In [48]:
def return_latest():
    df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
    X = df.iloc[:,0:21]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    return X[-1]

In [49]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='tanh'))

generator.compile(loss=my_loss, optimizer='adam')

In [50]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    loss = my_loss(noise, X_pred)

    pop = dnn_model.predict(X_pred, verbose=0)
    generator.train_on_batch(noise, X_pred)
    
    result.append(X_pred)
    pop_result.append(pop)

    if i % 10 == 0:
        print(f"epoch: {i}, loss: {loss}")

epoch: 0, loss: [[39.971718]]
epoch: 10, loss: [[38.992382]]
epoch: 20, loss: [[36.646557]]
epoch: 30, loss: [[32.12387]]
epoch: 40, loss: [[28.517868]]
epoch: 50, loss: [[27.388714]]
epoch: 60, loss: [[25.797562]]
epoch: 70, loss: [[25.63386]]
epoch: 80, loss: [[24.803062]]
epoch: 90, loss: [[24.497356]]
epoch: 100, loss: [[24.563301]]
epoch: 110, loss: [[24.27842]]
epoch: 120, loss: [[24.216526]]
epoch: 130, loss: [[24.281258]]
epoch: 140, loss: [[24.206425]]
epoch: 150, loss: [[24.031727]]
epoch: 160, loss: [[24.32211]]
epoch: 170, loss: [[24.036713]]
epoch: 180, loss: [[24.068684]]
epoch: 190, loss: [[24.229069]]
epoch: 200, loss: [[24.202553]]
epoch: 210, loss: [[24.005543]]
epoch: 220, loss: [[24.160519]]
epoch: 230, loss: [[24.125542]]
epoch: 240, loss: [[24.173851]]
epoch: 250, loss: [[24.17445]]
epoch: 260, loss: [[24.05065]]
epoch: 270, loss: [[24.019905]]
epoch: 280, loss: [[23.98544]]
epoch: 290, loss: [[24.132898]]
epoch: 300, loss: [[23.993885]]
epoch: 310, loss: [[23.921

In [65]:
result = np.array(result).reshape(1000, 21)
pop_result = np.array(pop_result).reshape(1000, 1)
X_result = scaler.inverse_transform(result)

pd_result = pd.DataFrame(X_result)
pd_result.to_excel('basic.xlsx')

pd_pop_result = pd.DataFrame(pop_result)
pd_pop_result.to_excel('basic_pop.xlsx')